In [3]:
from bs4 import BeautifulSoup #スクレイピングのためのライブラリ
import requests #HTTP操作用
import re
import time

/Users/takahashikaisei/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
y = 2023
m = 12
url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0370&year={y}&month={m}'
r = requests.get(url)

In [5]:
r

<Response [200]>

In [17]:
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [7]:
print(soup.find('title').string)

気象庁｜過去の気象データ検索


In [11]:
tags = soup.find_all('table',class_ = 'data2_s')
tags

[<table class="data2_s" id="tablefix1">
 <tr class="mtx">
 <th rowspan="3" scope="col">日</th>
 <th colspan="3" scope="colgroup">降水量</th>
 <th colspan="3" scope="colgroup">気温</th>
 <th colspan="2" scope="colgroup">湿度</th>
 <th colspan="6" scope="colgroup">風向・風速</th>
 <th rowspan="3" scope="col">日照<br/>時間<br/>(h)</th>
 <th colspan="2" scope="colgroup">雪</th>
 </tr>
 <tr class="mtx">
 <th rowspan="2" scope="col">合計<br/>(mm)</th><th rowspan="2" scope="col">最大1時間<br/>(mm)</th><th rowspan="2" scope="col">最大10分間<br/>(mm)</th>
 <th rowspan="2" scope="col">平均<br/>(℃)</th><th rowspan="2" scope="col">最高<br/>(℃)</th><th rowspan="2" scope="col">最低<br/>(℃)</th>
 <th rowspan="2" scope="col">平均<br/>(％)</th><th rowspan="2" scope="col">最小<br/>(％)</th>
 <th rowspan="2" scope="col">平均風速<br/>(m/s)</th><th colspan="2" scope="colgroup">最大</th><th colspan="2" scope="colgroup">最大瞬間</th><th rowspan="2" scope="col">最多<br/>風向</th>
 <th rowspan="2" scope="col">降雪の深さの<br/>合計(cm)</th><th rowspan="2" scope="col">最深積雪

classをdata_0_0で絞れば撮りたいものはとれる

In [20]:
tags2 = soup.find('td',class_ = 'data_0_0').string
tags2

'0.0'

これをfor文で回して、データベースにいい感じに格納すればいい